# ADHDaptive: Thriving with ADHD

Our mission is to provide a reliable and empathetic virtual companion that offers personalized assistance, guidance, and understanding to help students with ADHD thrive academically and emotionally. Through this initiative, we aim to foster a more inclusive and supportive learning environment where every student has the opportunity to reach their full potential.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


from string import punctuation
import re

In [ ]:
data = pd.read_csv('/content/ADHD.csv')
data = data.sample(frac=0.1, random_state=42)
data.dataframeName = 'Mental_Health_FAQ.csv'
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 33607 rows and 8 columns


In [ ]:
data.head()

,title,selftext,score,id,url,num_comments,created_utc,created_datetime
329995,Alcohol and atomoxetine,[removed],1,kzq09o,https://www.reddit.com/r/ADHD/comments/kzq09o/...,1.0,1610955536.0,2021-01-18 07:38:56
55953,Could use some thoughts/opinions (sorry its lo...,[deleted],0,4y6luk,https://www.reddit.com/r/ADHD/comments/4y6luk/...,0.0,1471451707.0,2016-08-17 16:35:07
188686,"Irresponsible thing that I did? Bills, bills, ...","So, today I cleaned up my credit card bills. I...",2,cnwest,https://www.reddit.com/r/ADHD/comments/cnwest/...,1.0,1565319907.0,2019-08-09 03:05:07
96563,Anyone take antidepressants in combination or ...,[deleted],8,7o5c83,https://www.reddit.com/r/ADHD/comments/7o5c83/...,22.0,1515093778.0,2018-01-04 19:22:58
188300,I need help with studying for important tests,"At the moment, I’m heading to year 11 which is...",2,cn61rk,https://www.reddit.com/r/ADHD/comments/cn61rk/...,4.0,1565185855.0,2019-08-07 13:50:55


In [ ]:
data['Questions'] = data['title'].str.lower()
data['Questions'] = data['title'].str.replace('[^\w\s]', '')
data.dropna(inplace=True)
data['Answers'] = data['selftext'].str.lower()
data['Answers'] = data['selftext'].str.replace('[^\w\s]', '')
data['Answers'] = data['selftext'].str.replace('\n', '')
data['Answers'] = data['selftext'].str.replace('/r/', '')
data['Answers'] = data['selftext'].str.replace('/r/', '')
data.dropna(inplace=True)

In [ ]:
data = data[~data.apply(lambda row: '[removed]' in row.values, axis=1)]
data = data[~data.apply(lambda row: '[deleted]' in row.values, axis=1)]

In [ ]:
data

,title,selftext,score,id,url,num_comments,created_utc,created_datetime,Questions,Answers
188686,"Irresponsible thing that I did? Bills, bills, ...","So, today I cleaned up my credit card bills. I...",2,cnwest,https://www.reddit.com/r/ADHD/comments/cnwest/...,1.0,1565319907.0,2019-08-09 03:05:07,Irresponsible thing that I did Bills bills bills,"So, today I cleaned up my credit card bills. I..."
188300,I need help with studying for important tests,"At the moment, I’m heading to year 11 which is...",2,cn61rk,https://www.reddit.com/r/ADHD/comments/cn61rk/...,4.0,1565185855.0,2019-08-07 13:50:55,I need help with studying for important tests,"At the moment, I’m heading to year 11 which is..."
24373,issues with my mood and anxiety whilst medicated,any1 have any tips for boosting my general moo...,1,2kk8k4,https://www.reddit.com/r/ADHD/comments/2kk8k4/...,0.0,1414498776.0,2014-10-28 12:19:36,issues with my mood and anxiety whilst medicated,any1 have any tips for boosting my general moo...
106869,Finally found a good generic Adderall (uninsured),So as many other insured people are probably a...,20,8aijf4,https://www.reddit.com/r/ADHD/comments/8aijf4/...,21.0,1523112868.0,2018-04-07 14:54:28,Finally found a good generic Adderall uninsured,So as many other insured people are probably a...
115104,Setting myself a self-care regimen for the nex...,I've been having significant mental health iss...,2,8p4dzj,https://www.reddit.com/r/ADHD/comments/8p4dzj/...,2.0,1528319182.0,2018-06-06 21:06:22,Setting myself a selfcare regimen for the next...,I've been having significant mental health iss...
...,...,...,...,...,...,...,...,...,...,...
31029,Buying healthcare?,I'm buying healthcare sometime in the next mon...,1,31bv6q,https://www.reddit.com/r/ADHD/comments/31bv6q/...,1.0,1428081472.0,2015-04-03 17:17:52,Buying healthcare,I'm buying healthcare sometime in the next mon...
65959,"Now that I've been diagnosed, I am realizing m...","Today, I was supposed to go to work, leave e...",11,5mazn8,https://www.reddit.com/r/ADHD/comments/5mazn8/...,12.0,1483670561.0,2017-01-06 02:42:41,Now that Ive been diagnosed I am realizing my ...,"Today, I was supposed to go to work, leave e..."
195403,"“Well, just try to remember next time”","Okay, then. So teach me. Tell, me how exactly ...",13,d2tadw,https://www.reddit.com/r/ADHD/comments/d2tadw/...,7.0,1568221134.0,2019-09-11 16:58:54,Well just try to remember next time,"Okay, then. So teach me. Tell, me how exactly ..."
99832,Is it worth getting diagnosed while being over...,Hello there. Sorry for my english if it's bad ...,8,7vuqx4,https://www.reddit.com/r/ADHD/comments/7vuqx4/...,13.0,1517992712.0,2018-02-07 08:38:32,Is it worth getting diagnosed while being over...,Hello there. Sorry for my english if it's bad ...


In [ ]:
data = data.reset_index(drop=True)

In [ ]:
intent_counts = data['Questions'].value_counts()
# fig = px.bar(x=intent_counts.index, y=intent_counts.values, labels={'x': 'Intents', 'y': 'Count'})
# fig.show()

In [ ]:
data['Intent'] = data['Questions']
questions_response_counts = data.groupby('Intent').size().reset_index(name='Count')

In [ ]:
# Calculate average number of questions per intent
avg_questions = data.groupby('Intent').size().mean()

# # Visualize average pattern count per intent
# fig = px.bar(x=questions_response_counts['Intent'], y=questions_response_counts['Count'],
#              labels={'x': 'Intents', 'y': 'Average Questions Count'})
# fig.show()

In [ ]:
# Intent Prediction Model
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['Questions'], data['Intent'], test_size=0.2, random_state=42)

In [ ]:
# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the model
model = LinearSVC()
model.fit(X_train_vec, y_train)

# Evaluate the model
y_pred = model.predict(X_test_vec)
report = classification_report(y_test, y_pred)
print(report)

                                                                                                                                                                                                                                                                                       precision    recall  f1-score   support

                                                                                                                                                                                                                                           ADHD vs Dog trying to change my bed sheets       0.00      0.00      0.00         1
                                                                                                                                                                                                                                                                  and another breakup       0.00      0.00      0.00         1
                                          

In [ ]:
# Visualize model performance
metrics = ['precision', 'recall', 'f1-score', 'support']
scores = classification_report(y_test, y_pred, output_dict=True)['weighted avg']

# fig = px.bar(x=metrics, y=[scores[metric] for metric in metrics], labels={'x': 'Metrics', 'y': 'Score'})
# fig.show()

In [ ]:
print("Stressed or Overthinking about ADHD? Gets answers based on real experiences")
print("Ask a question or enter 'quit' to exit.")

while True:
    user_input = input("User: ")

    if user_input.lower() == 'quit':
        print("Bot: Goodbye!")
        break

    # Vectorize user input
    user_input_vec = vectorizer.transform([user_input.lower()])

    # Predict the intent
    predicted_intent = model.predict(user_input_vec)[0]

    # Implement response generation mechanism based on predicted intent
    response = data[data['Questions'] == predicted_intent]['Answers'].values[0] if predicted_intent in data['Questions'].values else "I'm sorry, I don't have a response for that question."

    print("Bot: " + response)

Stressed or Overthinking about ADHD? Gets answers based on real experiences
Ask a question or enter 'quit' to exit.
User: Hey! Anyone struggling with directions?
Bot: In middle school, especially gym class, I had a hard time understanding the rules of the games we were playing. I remember getting yelled at a lot and always being chosen last by team captain. I remember quitting a game and wandering off and walking around a tree. These memories make me sad because I was so lonely and I felt so dumb. I was book smart from 7th grade and up but I've never been logically smart. There was a time in JROTC where we had to do a project of connecting tubes together to try to get to a targeted spot without touching the floor. I kind of sat back and did nothing and this boy yelled at me like "come on, weren't you paying attention?" And I just dropped the class because it was for leaders, and I had anxiety. Also when we practiced marching in the class and we had to put the tip of our hands on one an